In [ ]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 4.0 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619433 sha256=04005e6cb4b431daa8e6b377498c679628f776b772714b0617289d5c9ceebe49
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
import csv
import json
import gzip
import pandas as pd
import numpy as np
import urllib.parse
import urllib.request
import urllib.error
import requests
import random
from collections import defaultdict
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from surprise import SVD, Reader, Dataset, accuracy
from surprise.model_selection import train_test_split
from surprise import NormalPredictor
from surprise.model_selection import GridSearchCV

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Reading the Video Games Ratings from the Amazon Review Data

---




In [ ]:
def readGz(path):
  for l in gzip.open(path, 'rt'):
    yield eval(l)

In [ ]:
metadata = list(readGz("/content/drive/MyDrive/CSE258_Assignment2/meta_Video_Games.json.gz"))
file = open("/content/drive/MyDrive/CSE258_Assignment2/Video_Games.csv")
csvreader = csv.reader(file)
dataset = []
for row in csvreader:
  dataset.append(row)

In [ ]:
df = pd.DataFrame(np.array(dataset), columns=['asin', 'user_id', 'rating', 'unix_time'])

In [ ]:
df.head

<bound method NDFrame.head of                asin         user_id rating   unix_time
0        0439381673  A21ROB4YDOZA5P    1.0  1402272000
1        0439381673  A3TNZ2Q5E7HTHD    3.0  1399680000
2        0439381673  A1OKRM3QFEATQO    4.0  1391731200
3        0439381673  A2XO1JFCNEYV3T    1.0  1391731200
4        0439381673  A19WLPIRHD15TH    4.0  1389830400
...             ...             ...    ...         ...
2565344  B01HJEBIAA   ANGB54K3888S4    5.0  1533081600
2565345  B01HJEBIAA  A3TEVKR0ZVQB2T    5.0  1531785600
2565346  B01HJEBIAA   ABE7YPWEHNVJZ    5.0  1530835200
2565347  B01HJEBIAA  A3ES9QBK3G192O    5.0  1528761600
2565348  B01HJEBIAA  A194UPGR3OXZB7    5.0  1524096000

[2565349 rows x 4 columns]>

In [ ]:
asin_map = {}
for meta in metadata:
  asin_map[meta['asin']] = meta['title']

Obtaining Global Data for the Dataset


In [ ]:
# Calculating the Mean of the Video Game Ratings
df['rating'] = df['rating'].astype(float)
print("The Global Average of the Video Game Ratings:", df['rating'].mean())

The Global Average of the Video Game Ratings: 4.0220948494727224


In [ ]:
# Obtaining the number of unique values
nAsins   = len(pd.unique(df['asin']))  
nUserIds = len(pd.unique(df['user_id']))

print("Number of unique ASINs stored:", nAsins)
print("Number of unique Users stored:", nUserIds)

Number of unique ASINs stored: 71982
Number of unique Users stored: 1540618


In [ ]:
# Calculating the standard deviation of the Video Game Ratings
df['rating'] = df['rating'].astype(float)
print("The Global standard deviation of the Video Game Ratings:", df['rating'].std())

The Global standard deviation of the Video Game Ratings: 1.405936004591162


In [ ]:
# Showing the distribution of ratings
df['rating'].value_counts(normalize=True)

5.0    0.579791
4.0    0.160763
1.0    0.121578
3.0    0.082775
2.0    0.055093
Name: rating, dtype: float64

In [ ]:
# Finding the most popular items
pop=df['asin'].value_counts()[:10]

In [ ]:
pop

B00178630A    7630
B00HTK1NCS    6462
B004RMK57U    5135
B00KKAQYXM    4359
B00JJNQG98    3962
B003ZSP0WW    3960
B000ZKA0J6    3930
B000VJRU44    3634
B00E4MQODC    3520
B000B9RI14    3345
Name: asin, dtype: int64

In [ ]:
for i,p in zip(pop.index,pop):
  print(p,asin_map[i])

7630 Diablo III
6462 Redragon M601 Wired Gaming Mouse, Ergonomic, Programmable 6 Buttons, 3200 DPI with Red LED Mouse for Windows PC Games - Black
5135 Playstation Plus: 3 Month Membership [Digital Code]
4359 HAVIT HV-MS672 3200DPI Wired Mouse, 4 Adjustable DPI Levels, 800/1200/2400/3200DPI, 7 Circular &amp; Breathing LED Light, 6 Buttons (Black)(Updated Version)
3962 HyperX Cloud Gaming Headset for PC, Xbox One&sup1;, PS4, PS4 PRO, Xbox One S&sup1;, Nnintedo Switch (KHX-H3CL/WR) - Black
3960 Xbox 360 Wireless Controller - Glossy Black
3930 StarCraft II: Wings of Liberty
3634 Wii Fit Game with Balance Board
3520 Logitech G602 Lag-Free Wireless Gaming Mouse &ndash; 11 Programmable Buttons, Up to 2500 DPI
3345 Xbox 360 LIVE 1600 Points


Similarity Functions

In [ ]:
usersPerItem = defaultdict(set) # Maps an item to the users who rated it
itemsPerUser = defaultdict(set) # Maps a user to the items that they rated
ratingDict = {} # To retrieve a rating for a specific user/item pair
timestampDict = {}
print(len(df))
for index, d in tqdm(df.iterrows()):
    user,item = d['user_id'], d['asin']
    usersPerItem[item].add(user)
    itemsPerUser[user].add(item)
    timestampDict[(user, item)] = d['unix_time']
    ratingDict[(user,item)] = d['rating']

2565349


0it [00:00, ?it/s]

In [ ]:
userAverages = {}
itemAverages = {}

for u in itemsPerUser:
    rs = [ratingDict[(u,i)] for i in itemsPerUser[u]]
    userAverages[u] = sum(rs) / len(rs)
    
for i in usersPerItem:
    rs = [ratingDict[(u,i)] for u in usersPerItem[i]]
    itemAverages[i] = sum(rs) / len(rs)

In [ ]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

In [ ]:
def mostSimilarFast(i, N):
    similarities = []
    users = usersPerItem[i]
    candidateItems = set()
    for u in users:
        candidateItems = candidateItems.union(itemsPerUser[u])
    for i2 in candidateItems:
        if i2 == i: continue
        sim = Jaccard(users, usersPerItem[i2])
        similarities.append((sim,i2))
    similarities.sort(reverse=True)
    return similarities[:N]

In [ ]:
query = df['asin'][0]
print(asin_map[query])

BRAND NEW Learning Company Oregon Trail 5th Edition Jewel Case Swollen Dreams Fears Adventures Rivers


In [ ]:
top10SimilarGames = mostSimilarFast(query, 10)

In [ ]:
for similarGame in top10SimilarGames:
  print(similarGame[0], asin_map[similarGame[1]])

0.02 Secret Mission: The Forgotten Island (PC DVD) (UK IMPORT)
0.02 Official Nintendo Game Traveler Case-Aqua Blue (3DS, DSi and DS Lite)
0.02 Sim Classics 3: Sim City, Sim Tower, Sim Isle
0.0196078431372549 ATV Mudracer [Download]
0.019230769230769232 Legends Platinum 10-Pack PC
0.019230769230769232 Mysteries of Magic Island (PC CD) (UK IMPORT)
0.019230769230769232 SeaWorld Adventure Parks Shamu's Deep Sea Adventure - Game Boy Advance
0.018518518518518517 Oregon Trail 2nd Edition
0.01818181818181818 Match 3 Quest (4-pack)
0.01818181818181818 Pajama Sam 2: Thunder and Lightning Aren&rsquo;t So Frightening [Download]


Code for Recommending New Video Game products to users


In [ ]:
# Bought/not bought predictions for Video Game Items
df = pd.DataFrame(np.array(dataset), columns=['asin', 'user_id', 'rating', 'unix_time'])
df = df.sample(frac=1).reset_index(drop=True)
train = df.iloc[:750000].to_dict('records')
valid = df.iloc[750000:1000000].to_dict('records')
test = df.iloc[1000000:].to_dict('records')

In [ ]:
# Create new dictionaries to hold information that ties users to items and items to users
usersPerItem=defaultdict(set)
itemsPerUser=defaultdict(set)
for t in train:
    u=t['user_id']
    i=t['asin']
    itemsPerUser[u].add(i)
    usersPerItem[i].add(u)

In [ ]:
count=[]
items=[]
for i in usersPerItem:
    items.append(i)
    count.append(len(usersPerItem[i]))

In [ ]:
mostPopularItems = pd.DataFrame({'item':items,'count':count})
mostPopularItems.sort_values(by='count',ascending=False,inplace=True)

In [ ]:
usersValid=[]
itemsValid=[]
boughtValid=[]
for v in valid:
    usersValid.append(v['user_id'])
    itemsValid.append(v['asin'])
    boughtValid.append(1)

In [ ]:
# usersPerItem=defaultdict(set)
# itemsPerUser=defaultdict(set)
# for t in train+valid:
#     u=t['user_id']
#     i=t['asin']
#     itemsPerUser[u].add(i)
#     usersPerItem[i].add(u)

In [ ]:
uniqueItems=np.unique(np.array(list(usersPerItem.keys())))

In [ ]:
# Creating the negative set for the Model
itemsValidNeg=[]
boughtValidNeg=[]
for u in usersValid:
    temp=random.choice(uniqueItems)
    while temp in itemsPerUser[u]:
        temp=random.choice(uniqueItems)
    itemsValidNeg.append(temp)
    boughtValidNeg.append(0)

In [ ]:
usersValid=usersValid+usersValid
itemsValid=itemsValid+itemsValidNeg
boughtValid=boughtValid+boughtValidNeg

Recommending Items based off of Popularality:

In [ ]:
def model1(threshold):
  result=[]
  for t in threshold:
    return1=set(mostPopularItems[:int(mostPopularItems.shape[0]/t)].item)
    count=0
    p1=0
    p0=0
    for i in range(len(usersValid)):
        if itemsValid[i] in return1:
            pred=1
        else:
            pred=0
        if pred==boughtValid[i]:
            count+=1
        if pred==1:
          p1+=1
        else:
          p0+=1

    result.append((count/len(usersValid),count,p0,p1,t))
  result.sort()
  print(result)
  return result

In [ ]:
import time
start = time.time() 
model1([5.8])# runtime: 0s
print("time :", time.time() - start)

[(0.804654, 402327, 261243, 238757, 5.8)]
time : 0.31530165672302246


In [ ]:
start = time.time() 
model1([2])# runtime: 0s
print("time :", time.time() - start)

[(0.716266, 358133, 141879, 358121, 2)]
time : 0.3650858402252197


In [ ]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

In [ ]:
def mostSimilar(r1,r2):
    return Jaccard(usersPerItem[r1],usersPerItem[r2])

In [ ]:
def model2(m,r):
  result=[]
  for para1,para2 in zip(m,r):
    predQ4=[]
    return1=set(mostPopularItems[:int(mostPopularItems.shape[0]/para2)].item)
    for u,r1 in zip(usersValid,itemsValid):
        if len(usersPerItem[r1])==0:
            if r1 in return1:
                predQ4.append(1)
            else:
                predQ4.append(0)
        else:
            max=0
            for r2 in itemsPerUser[u]:
                if mostSimilar(r1,r2) >max:
                    max=mostSimilar(r1,r2)
            if max>para1:
                predQ4.append(1)
            else:
                if r1 in return1:
                    predQ4.append(1)
                else:
                    predQ4.append(0)
    count=0
    p1=0
    p0=0
    for train,pred in zip(boughtValid,predQ4):
        if pred==0:
          p0+=1
        else:
          p1+=1
        if train==pred:
            count+=1
    result.append((count/500000,p0,p1,para1,para2))
  result.sort()
  print(result)

In [ ]:
#import time
start = time.time() 
model2(np.arange(0.05,0.06,0.01),np.arange(5.5,5.6,0.1)) # runtime: 10s
print("time :", time.time() - start)

[(0.805828, 256036, 243964, 0.05, 5.5)]
time : 12.586409091949463


Jaccard+Popular Items:

In [ ]:
# count=0
# p1=0
# p0=0
# for train,pred in zip(boughtValid,predQ4):
#     if pred==0:
#       p0+=1
#     else:
#       p1+=1
#     if train==pred:
#         count+=1
# print(count,p0,p1,count/500000)


In [ ]:
#0.807672

In [ ]:
# Applying to test set

In [ ]:
df = pd.DataFrame(np.array(dataset), columns=['asin', 'user_id', 'rating', 'unix_time'])

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
train=df.iloc[:1000000].to_dict('records')
test=df.iloc[1000000:].to_dict('records')

In [ ]:
usersPerItem=defaultdict(set)
itemsPerUser=defaultdict(set)
for t in train:
    u=t['user_id']
    i=t['asin']
    itemsPerUser[u].add(i)
    usersPerItem[i].add(u)

In [ ]:
count=[]
items=[]
for i in usersPerItem:
    items.append(i)
    count.append(len(usersPerItem[i]))

In [ ]:
mostPopularItems=pd.DataFrame({'item':items,'count':count})
mostPopularItems.sort_values(by='count',ascending=False,inplace=True)

In [ ]:
usersTest=[]
itemsTest=[]
boughtTest=[]
for t in test:
    usersTest.append(t['user_id'])
    itemsTest.append(t['asin'])
    boughtTest.append(1)

In [ ]:
uniqueItems=np.unique(np.array(list(usersPerItem.keys())))

In [ ]:
itemsTestNeg=[]
boughtTestNeg=[]
for u in usersTest:
    temp=random.choice(uniqueItems)
    while temp in itemsPerUser[u]:
        temp=random.choice(uniqueItems)
    itemsTestNeg.append(temp)
    boughtTestNeg.append(0)

In [ ]:
usersTest=usersTest+usersTest
itemsTest=itemsTest+itemsTestNeg
boughtTest=boughtTest+boughtTestNeg

In [ ]:
predQ4=[]
return1=set(mostPopularItems[:int(mostPopularItems.shape[0]/5.5)].item)
for u,r1 in zip(usersTest,itemsTest):
    if len(usersPerItem[r1])==0:
        if r1 in return1:
            predQ4.append(1)
        else:
            predQ4.append(0)
    else:
        max=0
        for r2 in itemsPerUser[u]:
            if mostSimilar(r1,r2) >max:
                max=mostSimilar(r1,r2)
        if max>0.05:
            predQ4.append(1)
        else:
            if r1 in return1:
                predQ4.append(1)
            else:
                predQ4.append(0)

In [ ]:
count=0
p0=0
p1=0
for train,pred in zip(boughtTest,predQ4):
    if train==pred:
        count+=1
    if pred==1:
      p1+=1
    else:
      p0+=1
print(count,p0,p1,count/len(boughtTest))

2549379 1578372 1552326 0.8143164878886434


In [ ]:
len(test)

1565349

# Code that is no longer in use

In [ ]:
#I don't know why but I cannot run for loop by using pandas dataframe. So I create dictionary.


allRatings = df.to_dict('records')
userSet = set()
itemSet = set()
buySet = set()

for d in allRatings:
    userSet.add(d['user_id'])
    itemSet.add(d['asin'])
    buySet.add((d['user_id'],d['asin']))


In [ ]:
df2 = df.sample(frac = 1).reset_index()
df2

,index,asin,user_id,rating,unix_time
0,694019,B0053BG370,A3HP5XBZIBOVXM,3.0,1363219200
1,2303300,B003NSLGW2,AGNOXS0B4HK9A,5.0,1483315200
2,694188,B000B43OY4,ACW1Q5ZX1A31W,5.0,1135036800
3,89546,B00PV2S6EE,A2G76UOW1TO64M,5.0,1438128000
4,2231838,B0016BVY7U,A1GD1G8XXPRYQ8,1.0,1243641600
...,...,...,...,...,...
2565344,2214528,B001B1W3GG,A3UCQTQHKCSV16,5.0,1447804800
2565345,1946637,B0073ESYZO,A3EW46QE5T7VUO,5.0,1409356800
2565346,726051,B005V0TU1O,AA78TCLHXPJOK,5.0,1412553600
2565347,1061922,B000XJNTNS,A31OPHKP9PEI2Y,5.0,1453852800


In [ ]:
len(df2)

2565349

In [ ]:
# I set Train and valid based on rating prediction model
df2 = df.sample(frac = 1) # if we do not shuffle, it shows very low accuracy
ratingsValid = df2[750000:1000000]
ratingsValid = ratingsValid.to_dict('records')
ratingsTrain = df2[:750000]
ratingsTrain = ratingsTrain.to_dict('records')

In [ ]:
lUserSet = list(userSet)
lItemSet = list(itemSet)

notBuy = set()
for d in ratingsValid:
    #u = random.choice(lUserSet)
    i = random.choice(lItemSet)
    if (d['user_id'],d['asin']) in buySet or (d['user_id'],d['asin']) in notBuy: continue
    notBuy.add((d['user_id'],d['asin']))

buyValid = set()
for d in ratingsValid:
    buyValid.add((d['user_id'],d['asin']))

In [ ]:
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
for d in ratingsTrain:
    r = float(d['rating'])
    ratingsPerUser[d['user_id']].append((d['asin'],d['rating']))
    ratingsPerItem[d['asin']].append((d['user_id'],d['rating']))

In [ ]:
# Need for loop or whatever that we can find the ideal parameter
correct = 0
p0, p1 = 0,0
for (label,sample) in [(1, buyValid), (0, notBuy)]:
    for (u,b) in sample:
        maxSim = 0
        users = set(ratingsPerItem[b])
        for b2,_ in ratingsPerUser[u]:
            sim = Jaccard(users,set(ratingsPerItem[b2]))
            if sim > maxSim:
                maxSim = sim
        pred = 0
        if maxSim > 0.025 or len(ratingsPerItem[b]) > 60: #those number should be changed
            pred = 1
            p1 += 1
        else:
            p0 += 1
        if pred == label:
            correct += 1

In [ ]:
correct, p0, p1   #by using correct, we need to find a set of parameters that makes the largest value
# need to modified, it is impossible that correct = p1

(135154, 114115, 135154)

In [ ]:
reader = Reader(rating_scale=(1.0, 5.0))
trainingData = Dataset.load_from_df(df[['asin', 'user_id', 'rating'][:1000000]], reader)
trainset, testset = train_test_split(trainingData, test_size=.25)
validationData = Dataset.load_from_df(df[['asin', 'user_id', 'rating'][1000000:]], reader)

In [ ]:
param_grid = {'n_factors': [5, 25, 50, 75, 100],'n_epochs': [20, 25, 30], 'lr_all': [0.0015,0.002,0.0025,0.005],'reg_all':[0.05,0.065, 0.070, 0.075]}
gs = GridSearchCV(SVD, param_grid, measures=['mse'], cv=3)
gs.fit(data)
print(gs.best_params['mse'])

In [ ]:
model = SVD()

In [ ]:
model.fit(trainset)
predictions = model.test(testset)

In [ ]:
accuracy.mse(predictions, verbose=True)

MSE: 1.6339


1.633882794975756

In [ ]:
# This code does not work. Please use it as reference
predictions = open("HWpredictions_Rated.txt", 'w')
for l in open("stub_Rated.txt"):
    if l.startswith("user_id"):
        predictions.write(l)
        continue
    u,b = l.strip().split('-')
    try:
      innerUID = algo.trainset.to_inner_uid(u)
    except ValueError:
      innerUID = None
    
    try:
      innerIID = algo.trainset.to_inner_iid(b)
    except ValueError:
      innerIID = None
    bu = 0
    bi = 0
    if innerUID != None:
        bu = algo.bu[innerUID]
    if innerIID != None:
        bi = algo.bi[innerIID]
    _ = predictions.write(u + '-' + b + ',' + str(alpha + bu + bi) + '\n')

predictions.close()